# Hierarchical construction with stk

by: Andrew Tarzia

### Update 25/02/24

Installation of py3Dmol using `pip install py3Dmol` is required.

# Imports 

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
from itertools import product
import subprocess
import time
import stk
import stko
import spindry as spd
%matplotlib inline

In [ ]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

# Working

In [ ]:
bb1 = stk.BuildingBlock('NCCN', [stk.PrimaryAminoFactory()])
bb2 = stk.BuildingBlock(
    'C1(=CC=C(C=C1N)Br)Br', [stk.BromoFactory()]
)
macrocycle = stk.ConstructedMolecule(
    topology_graph=stk.macrocycle.Macrocycle(
        building_blocks=(bb1, bb2),
        repeating_unit='AB',
        num_repeating_units=3,
        orientations=(0, 0.5),
        optimizer=stk.MCHammer(target_bond_length=3)
    ),
)

In [ ]:
show_stk_mol(macrocycle)

In [ ]:
bb1 = stk.BuildingBlock('NCCN', [stk.PrimaryAminoFactory()])
bb2 = stk.BuildingBlock('C1C(CC1N)N', [stk.PrimaryAminoFactory()])
bb5 = stk.BuildingBlock(
    'C1(=CC=C(C=C1N)Br)Br', [stk.BromoFactory()]
)
bb6 = stk.BuildingBlock(
    'C(#CBr)Br', [stk.BromoFactory()]
)
cb3 = stk.BuildingBlock(
    'CBr', [stk.BromoFactory()]
)
cb4 = stk.BuildingBlock(
    'C1(=CC=CC=C1)Br', [stk.BromoFactory()]
)

macrocycles = [macrocycle]
caps = [cb3, cb4]
linears_1 = [bb1, bb2]
linears_2 = [bb5, bb6]

In [ ]:
show_stk_mol(cb3)

In [ ]:
show_stk_mol(bb1)

In [ ]:
import os
os.mkdir('hier_output')

In [ ]:
count = 0
for cap, l1, l2 in product(caps, linears_1, linears_2):
    # Build axle.
    polymer = stk.ConstructedMolecule(
        topology_graph=stk.polymer.Linear(
            building_blocks=(cap, l1, l2),
            repeating_unit='ABCBA',
            num_repeating_units=1,
            orientations=(0, 0, 0, 0, 0),
            optimizer=stk.MCHammer()
        ),
    )
    # Convert to building block.
    axle = stk.BuildingBlock.init_from_molecule(polymer)

    # Build all possible rotaxanes.
    for cycle in macrocycles:
        # Convert to building block assuming prior construction.
        cycle = stk.BuildingBlock.init_from_molecule(cycle)
        rotaxane = stk.ConstructedMolecule(
            topology_graph=stk.rotaxane.NRotaxane(
                axle=axle,
                cycles=(cycle, ),
                repeating_unit='A',
                num_repeating_units=1,
            ),
        )
        rotaxane.write(f'hier_output/rot_hier_{count}.mol')
        count += 1